In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &
!ollama pull phi4

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling fd7b6731c33c:   0% ▕▏  16 MB/9.1 GB                  pulling manifest 
pulling fd7b6731c33c:   1% ▕▏  92 MB/9.1 GB       

In [ ]:
!pip install ollama torch pynvml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import ollama
import time
import psutil
import torch
import pynvml
from typing import Dict

def ollama_chat(model, prompt):
    response = ollama.chat(
        model=model,
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response['message']['content'], {
        "total_tokens": len(response['message']['content'].split())  # rough token estimate
    }


def track_agent_run(agent_fn, input):
    start_time = time.time()
    print(f"After Start Memory Useage: {get_memory_usage_mb()} MB, BY AGENT {agent_fn.__name__}")
    response, metadata = agent_fn(**input)
    print(f"After End Memory Useage: {get_memory_usage_mb()} MB, BY AGENT {agent_fn.__name__}")
    end_time = time.time()

    elapsed_time = end_time - start_time
    tokens_used = metadata.get('total_tokens', estimate_tokens(response))

    print(f"--- Run Stats ---")
    print(f"Time taken: {elapsed_time:.2f} seconds")
    print(f"Tokens used: {tokens_used}")
    print(f"Response: {response}")
    print(f"-----------------\n")
    return response

def estimate_tokens(text):
    words = len(text.split())
    return int(words * 1.3)  # Approximate: avg 1.3 tokens per word

def get_memory_usage_mb() -> Dict[str, float]:
    # CPU memory for the current process
    process = psutil.Process()
    cpu_memory_mb = process.memory_info().rss / (1024 * 1024)  # in MB

    # CPU usage (%)
    cpu_usage_percent = psutil.cpu_percent(interval=0.1)

    # GPU memory and usage
    gpu_memory_mb = 0.0
    gpu_usage_percent = 0.0
    gpu_total_memory_mb = 0.0

    if torch.cuda.is_available():
        try:
            # Initialize NVIDIA Management Library
            pynvml.nvmlInit()
            device_count = pynvml.nvmlDeviceGetCount()

            if device_count > 0:
                # Get handle for the first GPU
                handle = pynvml.nvmlDeviceGetHandleByIndex(0)
                mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)

                # Total and used memory in MB
                gpu_total_memory_mb = mem_info.total / (1024 * 1024)
                gpu_used_memory_mb = mem_info.used / (1024 * 1024)

                # Memory used by this process (if possible)
                gpu_memory_mb = torch.cuda.memory_allocated() / (1024 * 1024)  # PyTorch-specific
                gpu_usage_percent = (gpu_used_memory_mb / gpu_total_memory_mb) * 100 if gpu_total_memory_mb > 0 else 0.0

                # Optionally, get process-specific GPU memory (requires elevated permissions)
                try:
                    processes = pynvml.nvmlDeviceGetComputeRunningProcesses(handle)
                    for proc in processes:
                        if proc.pid == process.pid:
                            gpu_memory_mb = proc.usedGpuMemory / (1024 * 1024)
                            gpu_usage_percent = (gpu_memory_mb / gpu_total_memory_mb) * 100
                            break
                except pynvml.NVMLError:
                    pass  # Skip if permission denied or not supported

            pynvml.nvmlShutdown()
        except Exception as e:
            print(f"Error accessing GPU info: {e}")
    else:
        print("CUDA is not available. No GPU detected.")

    return {
        "cpu_memory_mb": round(cpu_memory_mb, 2),
        "cpu_usage_percent": round(cpu_usage_percent, 2),
        "gpu_memory_mb": round(gpu_memory_mb, 2),
        "gpu_usage_percent": round(gpu_usage_percent, 2),
        "gpu_total_memory_mb": round(gpu_total_memory_mb, 2)
    }

def run_agent(agent_fn, input_text):
    start_time = time.time()
    print(f"After Start Memory Useage: {get_memory_usage_mb()} MB, BY AGENT {agent_fn.__name__}")
    response, metadata = agent_fn(input_text)
    print(f"After End Memory Useage: {get_memory_usage_mb()} MB, BY AGENT {agent_fn.__name__}")
    end_time = time.time()

    elapsed_time = end_time - start_time
    tokens_used = metadata.get('total_tokens', estimate_tokens(response))


    return {
        "response": response,
        "time": elapsed_time,
        "tokens": tokens_used
    }

def compare_agents(unoptimized_fn, optimized_fn, input_text, title="Optimization Comparison"):
    print(f"\n===== {title} =====")
    print(f"Input: {input_text}\n")

    print("🔴 Running Unoptimized Version...")
    unopt = run_agent(unoptimized_fn, input_text)

    print("\n🟢 Running Optimized Version...")
    opt = run_agent(optimized_fn, input_text)

    print("\n🔎 Results Comparison:")
    print(f"{'Metric':<20}{'Unoptimized':<20}{'Optimized'}")
    print(f"{'-'*60}")
    print(f"{'Time (s)':<20}{unopt['time']:.2f}{' ':<15}{opt['time']:.2f}")
    print(f"{'Tokens Used':<20}{unopt['tokens']:<20}{opt['tokens']}")
    print(f"{'Response':<20}{unopt['response'][:50]}{'...':<5}{opt['response'][:50]}...")
    print("="*60 + "\n")

    print(f"{'Response full Unoptimized:'} {unopt['response']}\n {'Response full Optimized:'} {opt['response']}")

# What Can Be Optimized?
- Prompt Clarity

- Input Size Management

- Context Window Usage

- Output Format Control

- Memory and State Management

- Function Calling Efficiency

- Fallback Strategies

- System Instruction Tuning

- Latency Reduction (early stopping, fast models)

# Optimization 1: Prompt Clarity

In [ ]:
def agent_unoptimized(input_text):
    prompt = f"Answer the question: {input_text}"
    return ollama_chat(model="phi4", prompt=prompt)

def agent_optimized(input_text):
    prompt = f"""You are a weather assistant.
Give a short (one sentence) current weather summary for the location: {input_text}.
Only mention temperature and conditions."""
    return ollama_chat(model="phi4", prompt=prompt)

compare_agents(agent_unoptimized, agent_optimized, "Weather Paris?", "Optimization 1: Prompt Clarity")




===== Optimization 1: Prompt Clarity =====
Input: Weather Paris?

🔴 Running Unoptimized Version...
After Start Memory Useage: {'cpu_memory_mb': 595.05, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 1.74, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized
> <ipython-input-17-084759f1b3a2>(8)agent_optimized()
      6     prompt = f"{input_text}"
      7     result = ollama_chat(model="phi4", prompt=prompt)
----> 8     if not result.strip():
      9         # fallback prompt
     10         prompt2 = f"Retry: {input_text}. Be concise."

ipdb> prompt
'I dont have a yellow pen. Now give me my yellow pen.'
ipdb> exit



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.11/bdb.py", line 361, in set_quit
    sys.settrace(None)



After End Memory Useage: {'cpu_memory_mb': 595.49, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.94, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized

🟢 Running Optimized Version...
After Start Memory Useage: {'cpu_memory_mb': 595.49, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.94, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized
> <ipython-input-17-084759f1b3a2>(8)agent_optimized()
      6     prompt = f"{input_text}"
      7     result = ollama_chat(model="phi4", prompt=prompt)
----> 8     if not result.strip():
      9         # fallback prompt
     10         prompt2 = f"Retry: {input_text}. Be concise."

ipdb> exit
After End Memory Useage: {'cpu_memory_mb': 595.49, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.94, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized

🔎 Results Comparison:
Metric              Unoptimized         Optimized
-------------------------------

# Optimization 2: Input Size Management

In [ ]:
# https://www.blindtextgenerator.com/
large_input = """
One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower arm towards the viewer. Gregor then turned to look out the window at the dull weather. Drops of rain could be heard hitting the pane, which made him feel quite sad. "How about if I sleep a little bit longer and forget all this nonsense", he thought, but that was something he was unable to do because he was used to sleeping on his right, and in his present state couldn't get into that position. However hard he threw himself onto his right, he always rolled back to where he was. He must have tried it a hundred times, shut his eyes so that he wouldn't have to look at the floundering legs, and only stopped when he began to feel a mild, dull pain there that he had never felt before. "Oh, God", he thought, "what a strenuous career it is that I've chosen! Travelling day in and day out. Doing business like this takes much more effort than doing your own business at home, and on top of that there's the curse of travelling, worries about making train connections, bad and irregular food, contact with different people all the time so that you can never get to know anyone or become friendly with them. It can all go to Hell!" He felt a slight itch up on his belly; pushed himself slowly up on his back towards the headboard so that he could lift his head better; found where the itch was, and saw that it was covered with lots of little white spots which he didn't know what to make of; and when he tried to feel the place with one of his legs he drew it quickly back because as soon as he touched it he was overcome by a cold shudder. He slid back into his former position. "Getting up early all the time", he thought, "it makes you stupid. You've got to get enough sleep. Other travelling salesmen live a life of luxury. For instance, whenever I go back to the guest house during the morning to copy out the contract, these gentlemen are always still sitting there eating their breakfasts. I ought to just try that with my boss; I'd get kicked out on the spot. But who knows, maybe that would be the best thing for me. If I didn't have my parents to think about I'd have given in my notice a long time ago, I'd have gone up to the boss and told him just what I think, tell him everything I would, let him know just what I feel. He'd fall right off his desk! And it's a funny sort of business to be sitting up there at your desk, talking down at your subordinates from up there, especially when you have to go right up close because the boss is hard of hearing. Well, there's still some hope; once I've got the money together to pay off my parents' debt to him - another five or six years I suppose - that's definitely what I'll do. That's when I'll make the big change. First of all though, I've got to get up, my train leaves at five." And he looked over at the alarm clock, ticking on the chest of drawers. "God in Heaven!" he thought. It was half past six and the hands were quietly moving forwards, it was even later than half past, more like quarter to seven. Had the alarm clock not rung? He could see from the bed that it had been set for four o'clock as it should have been; it certainly must have rung. Yes, but was it possible to quietly sleep through that furniture-rattling noise? True, he had not slept peacefully, but probably all the more deeply because of that. What should he do now? The next train went at seven; if he were to catch that he would have to rush like mad and the collection of samples was still not packed, and he did not at all feel particularly fresh and lively. And even if he did catch the train he would not avoid his boss's anger as the office assistant would have been there to see the five o'clock train go, he would have put in his report about Gregor's not being there a long time ago. The office assistant was the boss's man, spineless, and with no understanding. What about if he reported sick? But that would be extremely strained and suspicious as in fifteen years of service Gregor had never once yet been ill. His boss would certainly come round with the doctor from the medical insurance company, accuse his parents of having a lazy son, and accept the doctor's recommendation not to make any claim as the doctor believed that no-one was ever ill but that many were workshy. And what's more, would he have been entirely wrong in this case? Gregor did in fact, apart from excessive sleepiness after sleeping for so long, feel completely well and even felt much hungrier than usual. One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower arm towards the viewer. Gregor then turned to look out the window at the dull weather. Drops of rain could be heard hitting the pane, which made him feel quite sad. "How about if I sleep a little bit longer and forget all this nonsense", he thought, but that was something he was unable to do because he was used to sleeping on his right, and in his present state couldn't get into that position. However hard he threw himself onto his right, he always rolled back to where he was. He must have tried it a hundred times, shut his eyes so that he wouldn't have to look at the floundering legs, and only stopped when he began to feel a mild, dull pain there that he had never felt before. "Oh, God", he thought, "what a strenuous career it is that I've chosen! Travelling day in and day out. Doing business like this takes much more effort than doing your own business at home, and on top of that there's the curse of travelling, worries about making train connections, bad and irregular food, contact with different people all the time so that you can never get to know anyone or become friendly with them. It can all go to Hell!" He felt a slight itch up on his belly; pushed himself slowly up on his back towards the headboard so that he could lift his head better; found where the itch was, and saw that it was covered with lots of little white spots which he didn't know what to make of; and when he tried to feel the place with one of his legs he drew it quickly back because as soon as he touched it he was overcome by a cold shudder. He slid back into his former position. "Getting up early all the time", he thought, "it makes you stupid. You've got to get enough sleep. Other travelling salesmen live a life of luxury. For instance, whenever I go back to the guest house during the morning to copy out the contract, these gentlemen are always still sitting there eating their breakfasts. I ought to just try that with my boss; I'd get kicked out on the spot. But who knows, maybe that would be the best thing for me. If I didn't have my parents to think about I'd have given in my notice a long time ago, I'd have gone up to the boss and told him just what I think, tell him everything I would, let him know just what I feel. He'd fall right off his desk! And it's a funny sort of business to be sitting up there at your desk, talking down at your subordinates from up there, especially when you have to go right up close because the boss is hard of hearing. Well, there's still some hope; once I've got the money together to pay off my parents' debt to him - another five or six years I suppose - that's definitely what I'll do. That's when I'll make the big change. First of all though, I've got to get up, my train leaves at five." And he looked over at the alarm clock, ticking on the chest of drawers. "God in Heaven!" he thought. It was half past six and the hands were quietly moving forwards, it was even later than half past, more like quarter to seven. Had the alarm clock not rung? He could see from the bed that it had been set for four o'clock as it should have been; it certainly must have rung. Yes, but was it possible to quietly sleep through that furniture-rattling noise? True, he had not slept peacefully, but probably all the more deeply because of that. What should he do now? The next train went at seven; if he were to catch that he would have to rush like mad and the collection of samples was still not packed, and he did not at all feel particularly fresh and lively. And even if he did catch the train he would not avoid his boss's anger as the office assistant would have been there to see the five o'clock train go, he would have put in his report about Gregor's not being there a long time ago. The office assistant was the boss's man, spineless, and with no understanding. What about if he reported sick? But that would be extremely strained and suspicious as in fifteen years of service Gregor had never once yet been ill. His boss would certainly come round with the doctor from the medical insurance company, accuse his parents of having a lazy son, and accept the doctor's recommendation not to make any claim as the doctor believed that no-one was ever ill but that many were workshy. And what's more, would he have been entirely wrong in this case? Gregor did in fact, apart from excessive sleepiness after sleeping for so long, feel completely well and even felt much hungrier than usual. One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower arm towards the viewer. Gregor then turned to look out the window at the dull weather. Drops of rain could be heard hitting the pane, which made him feel quite sad. "How about if I sleep a little bit longer and forget all this nonsense", he thought, but that was something he was unable to do because he was used to sleeping on his right, and in his present state couldn't get into that position. However hard he threw himself onto his right, he always rolled back to where he was. He must have tried it a hundred times, shut his eyes so that he wouldn't have to look at the floundering legs, and only stopped when he began to feel a mild, dull pain there that he had never felt before. "Oh, God", he thought, "what a strenuous career it is that I've chosen! Travelling day in and day out. Doing business like this takes much more effort than doing your own business at home, and on top of that there's the curse of travelling, worries about making train connections, bad and irregular food, contact with different people all the time so that you can never get to know anyone or become friendly with them. It can all go to Hell!" He felt a slight itch up on his belly; pushed himself slowly up on his back towards the headboard so that he could lift his head better; found where the itch was, and saw that it was covered with lots of little white spots which he didn't know what to make of; and when he tried to feel the place with one of his legs he drew it quickly back because as soon as he touched it he was overcome by a cold shudder. He slid back into his former position. "Getting up early all the time", he thought, "it makes you stupid. You've got to get enough sleep. Other travelling salesmen live a life of luxury. For instance, whenever I go back to the guest house during the morning to copy out the contract, these gentlemen are always still sitting there eating their breakfasts. I ought to just try that with my boss; I'd get kicked out on the spot. But who knows, maybe that would be the best thing for me. If I didn't have my parents to think about I'd have given in my notice a long time ago, I'd have gone up to the boss and told him just what I think, tell him everything I would, let him know just what I feel. He'd fall right off his desk! And it's a funny sort of business to be sitting up there at your desk, talking down at your subordinates from up there, especially when you have to go right up close because the boss is hard of hearing. Well, there's still some hope; once I've got the money together to pay off my parents' debt to him - another five or six years I suppose - that's definitely what I'll do. That's when I'll make the big change. First of all though, I've got to get up, my train leaves at five." And he looked over at the alarm clock, ticking on the chest of drawers. "God in Heaven!" he thought. It was half past six and the hands were quietly moving forwards, it was even later than half past, more like quarter to seven. Had the alarm clock not rung? He could see from the bed that it had been set for four o'clock as it should have been; it certainly must have rung. Yes, but was it possible to quietly sleep through that furniture-rattling noise? True, he had not slept peacefully, but probably all the more deeply because of that. What should he do now? The next train went at seven; if he were to catch that he would have to rush like mad and the collection of samples was still not packed, and he did not at all feel particularly fresh and lively. And even if he did catch the train he would not avoid his boss's anger as the office assistant would have been there to see the five o'clock train go, he would have put in his report about Gregor's not being there a long time ago. The office assistant was the boss's man, spineless, and with no understanding. What about if he reported sick? But that would be extremely strained and suspicious as in fifteen years of service Gregor had never once yet been ill. His boss would certainly come round with the doctor from the medical insurance company, accuse his parents of having a lazy son, and accept the doctor's recommendation not to make any claim as the doctor believed that no-one was ever ill but that many were workshy. And what's more, would he have been entirely wrong in this case? Gregor did in fact, apart from excessive sleepiness after sleeping for so long, feel completely well and even felt much hungrier than usual. One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower arm towards the viewer. Gregor then turned to look out the window at the dull weather. Drops of rain could be heard hitting the pane, which made him feel quite sad. "How about if I sleep a little bit longer and forget all this nonsense", he thought, but that was something he was unable to do because he was used to sleeping on his right, and in his present state couldn't get into that position. However hard he threw himself onto his right, he always rolled back to where he was. He must have tried it a hundred times, shut his eyes so that he wouldn't have to look at the floundering legs, and only stopped when he began to feel a mild, dull pain there that he had never felt before. "Oh, God", he thought, "what a strenuous career it is that I've chosen! Travelling day in and day out. Doing business like this takes much more effort than doing your own business at home, and on top of that there's the curse of travelling, worries about making train connections, bad and irregular food, contact with different people all the time so that you can never get to know anyone or become friendly with them. It can all go to Hell!" He felt a slight itch up on his belly; pushed himself slowly up on his back towards the headboard so that he could lift his head better; found where the itch was, and saw that it was covered with lots of little white spots which he didn't know what to make of; and when he tried to feel the place with one of his legs he drew it quickly back because as soon as he touched it he was overcome by a cold shudder. He slid back into his former position. "Getting up early all the time", he thought, "it makes you stupid. You've got to get enough sleep. Other travelling salesmen live a life of luxury. For instance, whenever I go back to the guest house during the morning to copy out the contract, these gentlemen are always still sitting there eating their breakfasts. I ought to just try that with my boss; I'd get kicked out on the spot. But who knows, maybe that would be the best thing for me. If I didn't have my parents to think about I'd have given in my notice a long time ago, I'd have gone up to the boss and told him just what I think, tell him everything I would, let him know just what I feel. He'd fall right off his desk! And it's a funny sort of business to be sitting up there at your desk, talking down at your subordinates from up there, especially when you have to go right up close because the boss is hard of hearing. Well, there's still some hope; once I've got the money together to pay off my parents' debt to him - another five or six years I suppose - that's definitely what I'll do. That's when I'll make the big change. First of all though, I've got to get up, my train leaves at five." And he looked over at the alarm clock, ticking on the chest of drawers. "God in Heaven!" he thought. It was half past six and the hands were quietly moving forwards, it was even later than half past, more like quarter to seven. Had the alarm clock not rung? He could see from the bed that it had been set for four o'clock as it should have been; it certainly must have rung. Yes, but was it possible to quietly sleep through that furniture-rattling noise? True, he had not slept peacefully, but probably all the more deeply because of that. What should he do now? The next train went at seven; if he were to catch that he would have to rush like mad and the collection of samples was still not packed, and he did not at all feel particularly fresh and lively. And even if he did catch the train he would not avoid his boss's anger as the office assistant would have been there to see the five o'clock train go, he would have put in his report about Gregor's not being there a long time ago. The office assistant was the boss's man, spineless, and with no understanding. What about if he reported sick? But that would be extremely strained and suspicious as in fifteen years of service Gregor had never once yet been ill. His boss would certainly come round with the doctor from the medical insurance company, accuse his parents of having a lazy son, and accept the doctor's recommendation not to make any claim as the doctor believed that no-one was ever ill but that many were workshy. And what's more, would he have been entirely wrong in this case? Gregor did in fact, apart from excessive sleepiness after sleeping for so long, feel completely well and even felt much hungrier than usual. One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower arm towards the viewer. Gregor then turned to look out the window at the dull weather. Drops of rain could be heard hitting the pane, which made him feel quite sad. "How about if I sleep a little bit longer and forget all this nonsense", he thought, but that was something he was unable to do because he was used to sleeping on his right, and in his present state couldn't get into that position. However hard he threw himself onto his right, he always rolled back to where he was. He must have tried it a hundred times, shut his eyes so that he wouldn't have to look at the floundering legs, and only stopped when he began to feel a mild, dull pain there that he had never felt before. "Oh, God", he thought, "what a strenuous career it is that I've chosen! Travelling day in and day out. Doing business like this takes much more effort than doing your own business at home, and on top of that there's the curse of travelling, worries about making train connections, bad and irregular food, contact with different people all the time so that you can never get to know anyone or become friendly with them. It can all go to Hell!" He felt a slight itch up on his belly; pushed himself slowly up on his back towards the headboard so that he could lift his head better; found where the itch was, and saw that it was covered with lots of little white spots which he didn't know what to make of; and when he tried to feel the place with one of his legs he drew it quickly back because as soon as he touched it he was overcome by a cold shudder. He slid back into his former position. "Getting up early all the time", he thought, "it makes you stupid. You've got to get enough sleep. Other travelling salesmen live a life of luxury. For instance, whenever I go back to the guest house during the morning to copy out the contract, these gentlemen are always still sitting there eating their breakfasts. I ought to just try that with my boss; I'd get kicked out on the spot. But who knows, maybe that would be the best thing for me. If I didn't have my parents to think about I'd have given in my notice a long time ago, I'd have gone up to the boss and told him just what I think, tell him everything I would, let him know just what I feel. He'd fall right off his desk! And it's a funny sort of business to be sitting up there at your desk, talking down at your subordinates from up there, especially when you have to go right up close because the boss is hard of hearing. Well, there's still some hope; once I've got the money together to pay off my parents' debt to him - another five or six years I suppose - that's definitely what I'll do. That's when I'll make the big change. First of all though, I've got to get up, my train leaves at five." And he looked over at the alarm clock, ticking on the chest of drawers. "God in Heaven!" he thought. It was half past six and the hands were quietly moving forwards, it was even later than half past, more like quarter to seven. Had the alarm clock not rung? He could see from the bed that it had been set for four o'clock as it should have been; it certainly must have rung. Yes, but was it possible to quietly sleep through that furniture-rattling noise? True, he had not slept peacefully, but probably all the more deeply because of that. What should he do now? The next train went at seven; if he were to catch that he would have to rush like mad and the collection of samples was still not packed, and he did not at all feel particularly fresh and lively. And even if he did catch the train he would not avoid his boss's anger as the office assistant would have been there to see the five o'clock train go, he would have put in his report about Gregor's not being there a long time ago. The office assistant was the boss's man, spineless, and with no understanding. What about if he reported sick? But that would be extremely strained and suspicious as in fifteen years of service Gregor had never once yet been ill. His boss would certainly come round with the doctor from the medical insurance company, accuse his parents of having a lazy son, and accept the doctor's recommendation not to make any claim as the doctor believed that no-one was ever ill but that many were workshy. And what's more, would he have been entirely wrong in this case? Gregor did in fact, apart from excessive sleepiness after sleeping for so long, feel completely well and even felt much hungrier than usual. One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower arm towards the viewer. Gregor then turned to look out the window at the dull weather. Drops of rain could be heard hitting the pane, which made him feel quite sad. "How about if I sleep a little bit longer and forget all this nonsense", he thought, but that was something he was unable to do because he was used to sleeping on his right, and in his present state couldn't get into that position. However hard he threw himself onto his right, he always rolled back to where he was. He must have tried it a hundred times, shut his eyes so that he wouldn't have to look at the floundering legs, and only stopped when he began to feel a mild, dull pain there that he had never felt before. "Oh, God", he thought, "what a strenuous career it is that I've chosen! Travelling day in and day out. Doing business like this takes much more effort than doing your own business at home, and on top of that there's the curse of travelling, worries about making train connections, bad and irregular food, contact with different people all the time so that you can never get to know anyone or become friendly with them. It can all go to Hell!" He felt a slight itch up on his belly; pushed himself slowly up on his back towards the headboard so that he could lift his head better; found where the itch was, and saw that it was covered with lots of little white spots which he didn't know what to make of; and when he tried to feel the place with one of his legs he drew it quickly back because as soon as he touched it he was overcome by a cold shudder. He slid back into his former position. "Getting up early all the time", he thought, "it makes you stupid. You've got to get enough sleep. Other travelling salesmen live a life of luxury. For instance, whenever I go back to the guest house during the morning to copy out the contract, these gentlemen are always still sitting there eating their breakfasts. I ought to just try that with my boss; I'd get kicked out on the spot. But who knows, maybe that would be the best thing for me. If I didn't have my parents to think about I'd have given in my notice a long time ago, I'd have gone up to the boss and told him just what I think, tell him everything I would, let him know just what I feel. He'd fall right off his desk! And it's a funny sort of business to be sitting up there at your desk, talking down at your subordinates from up there, especially when you have to go right up close because the boss is hard of hearing. Well, there's still some hope; once I've got the money together to pay off my parents' debt to him - another five or six years I suppose - that's definitely what I'll do. That's when I'll make the big change. First of all though, I've got to get up, my train leaves at five." And he looked over at the alarm clock, ticking on the chest of drawers. "God in Heaven!" he thought. It was half past six and the hands were quietly moving forwards, it was even later than half past, more like quarter to seven. Had the alarm clock not rung? He could see from the bed that it had been set for four o'clock as it should have been; it certainly must have rung. Yes, but was it possible to quietly sleep through that furniture-rattling noise? True, he had not slept peacefully, but probably all the more deeply because of that. What should he do now? The next train went at seven; if he were to catch that he would have to rush like mad and the collection of samples was still not packed, and he did not at all feel particularly fresh and lively. And even if he did catch the train he would not avoid his boss's anger as the office assistant would have been there to see the five o'clock train go, he would have put in his report about Gregor's not being there a long time ago. The office assistant was the boss's man, spineless, and with no understanding. What about if he reported sick? But that would be extremely strained and suspicious as in fifteen years of service Gregor had never once yet been ill. His boss would certainly come round with the doctor from the medical insurance company, accuse his parents of having a lazy son, and accept the doctor's recommendation not to make any claim as the doctor believed that no-one was ever ill but that many were workshy. And what's more, would he have been entirely wrong in this case? Gregor did in fact, apart from excessive sleepiness after sleeping for so long, feel completely well and even felt much hungrier than usual. One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower arm towards the viewer. Gregor then turned to look out the window at the dull weather. Drops of rain could be heard hitting the pane, which made him feel quite sad. "How about if I sleep a little bit longer and forget all this nonsense", he thought, but that was something he was unable to do because he was used to sleeping on his right, and in his present state couldn't get into that position. However hard he threw himself onto his right, he always rolled back to where he was. He must have tried it a hundred times, shut his eyes so that he wouldn't have to look at the floundering legs, and only stopped when he began to feel a mild, dull pain there that he had never felt before. "Oh, God", he thought, "what a strenuous career it is that I've chosen! Travelling day in and day out. Doing business like this takes much more effort than doing your own business at home, and on top of that there's the curse of travelling, worries about making train connections, bad and irregular food, contact with different people all the time so that you can never get to know anyone or become friendly with them. It can all go to Hell!" He felt a slight itch up on his belly; pushed himself slowly up on his back towards the headboard so that he could lift his head better; found where the itch was, and saw that it was covered with lots of little white spots which he didn't know what to make of; and when he tried to feel the place with one of his legs he drew it quickly back because as soon as he touched it he was overcome by a cold shudder. He slid back into his former position. "Getting up early all the time", he thought, "it makes you stupid. You've got to get enough sleep. Other travelling salesmen live a life of luxury. For instance, whenever I go back to the guest house during the morning to copy out the contract, these gentlemen are always still sitting there eating their breakfasts. I ought to just try that with my boss; I'd get kicked out on the spot. But who knows, maybe that would be the best thing for me. If I didn't have my parents to think about I'd have given in my notice a long time ago, I'd have gone up to the boss and told him just what I think, tell him everything I would, let him know just what I feel. He'd fall right off his desk! And it's a funny sort of business to be sitting up there at your desk, talking down at your subordinates from up there, especially when you have to go right up close because the boss is hard of hearing. Well, there's still some hope; once I've got the money together to pay off my parents' debt to him - another five or six years I suppose - that's definitely what I'll do. That's when I'll make the big change. First of all though, I've got to get up, my train leaves at five." And he looked over at the alarm clock, ticking on the chest of drawers. "God in Heaven!" he thought. It was half past six and the hands were quietly moving forwards, it was even later than half past, more like quarter to seven. Had the alarm clock not rung? He could see from the bed that it had been set for four o'clock as it should have been; it certainly must have rung. Yes, but was it possible to quietly sleep through that furniture-rattling noise? True, he had not slept peacefully, but probably all the more deeply because of that. What should he do now? The next train went at seven; if he were to catch that he would have to rush like mad and the collection of samples was still not packed, and he did not at all feel particularly fresh and lively. And even if he did catch the train he would not avoid his boss's anger as the office assistant would have been there to see the five o'clock train go, he would have put in his report about Gregor's not being there a long time ago. The office assistant was the boss's man, spineless, and with no understanding. What about if he reported sick? But that would be extremely strained and suspicious as in fifteen years of service Gregor had never once yet been ill. His boss would certainly come round with the doctor from the medical insurance company, accuse his parents of having a lazy son, and accept the doctor's recommendation not to make any claim as the doctor believed that no-one was ever ill but that many were workshy. And what's more, would he have been entirely wrong in this case? Gregor did in fact, apart from excessive sleepiness after sleeping for so long, feel completely well and even felt much hungrier than usual. One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower arm towards the viewer. Gregor then turned to look out the window at the dull weather. Drops of rain could be heard hitting the pane, which made him feel quite sad. "How about if I sleep a little bit longer and forget all this nonsense", he thought, but that was something he was unable to do because he was used to sleeping on his right, and in his present state couldn't get into that position. However hard he threw himself onto his right, he always rolled back to where he was. He must have tried it a hundred times, shut his eyes so that he wouldn't have to look at the floundering legs, and only stopped when he began to feel a mild, dull pain there that he had never felt before. "Oh, God", he thought, "what a strenuous career it is that I've chosen! Travelling day in and day out. Doing business like this takes much more effort than doing your own business at home, and on top of that there's the curse of travelling, worries about making train connections, bad and irregular food, contact with different people all the time so that you can never get to know anyone or become friendly with them. It can all go to Hell!" He felt a slight itch up on his belly; pushed himself slowly up on his back towards the headboard so that he could lift his head better; found where the itch was, and saw that it was covered with lots of little white spots which he didn't know what to make of; and when he tried to feel the place with one of his legs he drew it quickly back because as soon as he touched it he was overcome by a cold shudder. He slid back into his former position. "Getting up early all the time", he thought, "it makes you stupid. You've got to get enough sleep. Other travelling salesmen live a life of luxury. For instance, whenever I go back to the guest house during the morning to copy out the contract, these gentlemen are always still sitting there eating their breakfasts. I ought to just try that with my boss; I'd get kicked out on the spot. But who knows, maybe that would be the best thing for me. If I didn't have my parents to think about I'd have given in my notice a long time ago, I'd have gone up to the boss and told him just what I think, tell him everything I would, let him know just what I feel. He'd fall right off his desk! And it's a funny sort of business to be sitting up there at your desk, talking down at your subordinates from up there, especially when you have to go right up close because the boss is hard of hearing. Well, there's still some hope; once I've got the money together to pay off my parents' debt to him - another five or six years I suppose - that's definitely what I'll do. That's when I'll make the big change. First of all though, I've got to get up, my train leaves at five." And he looked over at the alarm clock, ticking on the chest of drawers. "God in Heaven!" he thought. It was half past six and the hands were quietly moving forwards, it was even later than half past, more like quarter to seven. Had the alarm clock not rung? He could see from the bed that it had been set for four o'clock as it should have been; it certainly must have rung. Yes, but was it possible to quietly sleep through that furniture-rattling noise? True, he had not slept peacefully, but probably all the more deeply because of that. What should he do now? The next train went at seven; if he were to catch that he would have to rush like mad and the collection of samples was still not packed, and he did not at all feel particularly fresh and lively. And even if he did catch the train he would not avoid his boss's anger as the office assistant would have been there to see the five o'clock train go, he would have put in his report about Gregor's not being there a long time ago. The office assistant was the boss's man, spineless, and with no understanding. What about if he reported sick? But that would be extremely strained and suspicious as in fifteen years of service Gregor had never once yet been ill. His boss would certainly come round with the doctor from the medical insurance company, accuse his parents of having a lazy son, and accept the doctor's recommendation not to make any claim as the doctor believed that no-one was ever ill but that many were workshy. And what's more, would he have been entirely wrong in this case? Gregor did in fact, apart from excessive sleepiness after sleeping for so long, feel completely well and even felt much hungrier than usual. One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower arm towards the viewer. Gregor then turned to look out the window at the dull weather. Drops of rain could be heard hitting the pane, which made him feel quite sad. "How about if I sleep a little bit longer and forget all this nonsense", he thought, but that was something he was unable to do because he was used to sleeping on his right, and in his present state couldn't get into that position. However hard he threw himself onto his right, he always rolled back to where he was. He must have tried it a hundred times, shut his eyes so that he wouldn't have to look at the floundering legs, and only stopped when he began to feel a mild, dull pain there that he had never felt before. "Oh, God", he thought, "what a strenuous career it is that I've chosen! Travelling day in and day out. Doing business like this takes much more effort than doing your own business at home, and on top of that there's the curse of travelling, worries about making train connections, bad and irregular food, contact with different people all the time so that you can never get to know anyone or become friendly with them. It can all go to Hell!" He felt a slight itch up on his belly; pushed himself slowly up on his back towards the headboard so that he could lift his head better; found where the itch was, and saw that it was covered with lots of little white spots which he didn't know what to make of; and when he tried to feel the place with one of his legs he drew it quickly back because as soon as he touched it he was overcome by a cold shudder. He slid back into his former position. "Getting up early all the time", he thought, "it makes you stupid. You've got to get enough sleep. Other travelling salesmen live a life of luxury. For instance, whenever I go back to the guest house during the morning to copy out the contract, these gentlemen are always still sitting there eating their breakfasts. I ought to just try that with my boss; I'd get kicked out on the spot. But who knows, maybe that would be the best thing for me. If I didn't have my parents to think about I'd have given in my notice a long time ago, I'd have gone up to the boss and told him just what I think, tell him everything I would, let him know just what I feel. He'd fall right off his desk! And it's a funny sort of business to be sitting up there at your desk, talking down at your subordinates from up there, especially when you have to go right up close because the boss is hard of hearing. Well, there's still some hope; once I've got the money together to pay off my parents' debt to him - another five or six years I suppose - that's definitely what I'll do. That's when I'll make the big change. First of all though, I've got to get up, my train leaves at five." And he looked over at the alarm clock, ticking on the chest of drawers. "God in Heaven!" he thought. It was half past six and the hands were quietly moving forwards, it was even later than half past, more like quarter to seven. Had the alarm clock not rung? He could see from the bed that it had been set for four o'clock as it should have been; it certainly must have rung. Yes, but was it possible to quietly sleep through that furniture-rattling noise? True, he had not slept peacefully, but probably all the more deeply because of that. What should he do now? The next train went at seven; if he were to catch that he would have to rush like mad and the collection of samples was still not packed, and he did not at all feel particularly fresh and lively. And even if he did catch the train he would not avoid his boss's anger as the office assistant would have been there to see the five o'clock train go, he would have put in his report about Gregor's not being there a long time ago. The office assistant was the boss's man, spineless, and with no understanding. What about if he reported sick? But that would be extremely strained and suspicious as in fifteen years of service Gregor had never once yet been ill. His boss would certainly come round with the doctor from the medical insurance company, accuse his parents of having a lazy son, and accept the doctor's recommendation not to make any claim as the doctor believed that no-one was ever ill but that many were workshy. And what's more, would he have been entirely wrong in this case? Gregor did in fact, apart from excessive sleepiness after sleeping for so long, feel completely well and even felt much hungrier than usual.One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy fur muff that covered the whole of her lower arm towards the viewer. Gregor then turned to look out the window at the dull weather. Drops of rain could be heard hitting the pane, which made him feel quite sad. "How about if I sleep a little bit longer and forget all this nonsense", he thought, but that was something he was unable to do because he was used to sleeping on his right, and in his present state couldn't get into that position. However hard he threw himself onto his right, he always rolled back to where he was. He must have tried it a hundred times, shut his eyes so that he wouldn't have to look at the floundering legs, and only stopped when he began to feel a mild, dull pain there that he had never felt before. "Oh, God", he thought, "what a strenuous career it is that I've chosen! Travelling day in and day out. Doing business like this takes much more effort than doing your own business at home, and on top of that there's the curse of travelling, worries about making train connections, bad and irregular food, contact with different people all the time so that you can never get to know anyone or become friendly with them. It can all go to Hell!" He felt a slight itch up on his belly; pushed himself slowly up on his back towards the headboard so that he could lift his head better; found where the itch was, and saw that it was covered with lots of little white spots which he didn't know what to make of; and when he tried to feel the place with one of his legs he drew it quickly back because as soon as he touched it he was overcome by a cold shudder. He slid back into his former position. "Getting up early all the time", he thought, "it makes you stupid. You've got to get enough sleep. Other travelling salesmen live a life of luxury. For instance, whenever I go back to the guest house during the morning to copy out the contract, these gentlemen are always still sitting there eating their breakfasts. I ought to just try that with my boss; I'd get kicked out on the spot. But who knows, maybe that would be the best thing for me. If I didn't have my parents to think about I'd have given in my notice a long time ago, I'd have gone up to the boss and told him just what I think, tell him everything I would, let him know just what I feel. He'd fall right off his desk! And it's a funny sort of business to be sitting up there at your desk, talking down at your subordinates from up there, especially when you have to go right up close because the boss is hard of hearing. Well, there's still some hope; once I've got the money together to pay off my parents' debt to him - another five or six years I suppose - that's definitely what I'll do. That's when I'll make the big change. First of all though, I've got to get up, my train leaves at five." And he looked over at the alarm clock, ticking on the chest of drawers. "God in Heaven!" he thought. It was half past six and the hands were quietly moving forwards, it was even later than half past, more like quarter to seven. Had the alarm clock not rung? He could see from the bed that it had been set for four o'clock as it should have been; it certainly must have rung. Yes, but was it possible to quietly sleep through that furniture-rattling noise? True, he had not
"""

In [ ]:
def agent_unoptimized(big_input):
    prompt = f"Summarize this: {big_input}"
    return ollama_chat(model="phi4", prompt=prompt)

def agent_optimized(big_input):
    # Pre-truncate or summarize first few lines
    important_part = big_input[:2000]  # First 2000 characters
    prompt = f"Summarize this content: {important_part}"
    return ollama_chat(model="phi4", prompt=prompt)

compare_agents(agent_unoptimized, agent_optimized, large_input, "Optimization 2: Input Size Management")


===== Optimization 2: Input Size Management =====
Input: 
One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin. He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections. The bedding was hardly able to cover it and seemed ready to slide off any moment. His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked. "What's happened to me?" he thought. It wasn't a dream. His room, a proper human room although a little too small, lay peacefully between its four familiar walls. A collection of textile samples lay spread out on the table - Samsa was a travelling salesman - and above it there hung a picture that he had recently cut out of an illustrated magazine and housed in a nice, gilded frame. It showed a lady fitted out with a fur hat and fur boa who sat upright, raising a heavy

# Optimization 3: Context Window Usage

In [ ]:
history = ["Hi", "Tell me about cats", "Tell me about dogs", "Now lions"]

last_question_answer_pair = [{
  "question": "Now lions",
  "answer": """
  ### Lions:
- **Habitat**: Native to sub-Saharan Africa; a small population resides in India's Gir Forest.
- **Social Structure**: Known for their pride social structure, typically consisting of related females, cubs, and a few males.
- **Diet**: Carnivorous apex predators that hunt large ungulates like zebras and wildebeests.
- **Conservation Status**: Vulnerable due to habitat loss, human-wildlife conflict, and poaching.
"""
}]

context = "\n".join(history)

In [ ]:
def agent_unoptimized(new_question, history):
    context = "\n".join(history)
    prompt = f"{context}\nUser: {new_question}\nAnswer:"
    return ollama_chat(model="phi4", prompt=prompt)

def agent_optimized(new_question, last_question_answer_pair):
    context = f"Previous Q&A:\n{last_question_answer_pair}"
    prompt = f"{context}\nNew Question: {new_question}\nAnswer:"
    return ollama_chat(model="phi4", prompt=prompt)

track_agent_run(agent_unoptimized, {"new_question": "now tell me about rabbit", "history": history})
track_agent_run(agent_optimized, {"new_question": "now tell me about rabbit", "last_question_answer_pair": last_question_answer_pair})

After Start Memory Useage: {'cpu_memory_mb': 594.62, 'cpu_usage_percent': 9.5, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 1.74, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized
After End Memory Useage: {'cpu_memory_mb': 594.54, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.94, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized
--- Run Stats ---
Time taken: 61.57 seconds
Tokens used: 261
Response: Sure! Here's a brief overview of each animal:

### Cats:
Cats are small, typically furry carnivorous mammals. Domestic cats are often valued by humans for companionship and their ability to hunt vermin. They are known for being independent but also affectionate with their owners. Cats have flexible bodies, sharp retractable claws, and keen senses adapted for hunting.

### Dogs:
Dogs are domesticated mammals that are widely considered man's best friend. They come in a variety of breeds, sizes, and temperaments. Originally bred from wolves, dog

'### Rabbits:\n\n- **Habitat**: Rabbits are highly adaptable and can be found in a wide range of environments across the world, including forests, grasslands, deserts, and even urban areas. They prefer areas with ample cover such as bushes or tall grass to hide from predators.\n\n- **Social Structure**: Typically, rabbits live in groups known as warrens, which are complex systems of burrows dug into the ground. These social structures help them protect themselves from predators and raise their young collectively.\n\n- **Diet**: Rabbits are herbivores and have a diet primarily consisting of grasses, leaves, and vegetables. They practice coprophagy (eating their own fecal pellets) to extract more nutrients from their food.\n\n- **Reproduction**: Known for their high reproductive rate, rabbits can give birth to large litters several times a year. This rapid reproduction helps maintain their population despite predation pressures.\n\n- **Conservation Status**: Most rabbit species are not c

# Optimization 4: Output Format Control

In [ ]:
def agent_unoptimized(input_text):
    prompt = f"List the top 3 tourist spots in {input_text}."
    return ollama_chat(model="phi4", prompt=prompt)

def agent_optimized(input_text):
    prompt = f"""List the top 3 tourist spots in {input_text}.
Format it as:
1. Spot Name - Short Description
2. Spot Name - Short Description
3. Spot Name - Short Description"""
    return ollama_chat(model="phi4", prompt=prompt)


compare_agents(agent_unoptimized, agent_optimized, "Japan", "Optimization 4: Output Format Control")


===== Optimization 4: Output Format Control =====
Input: Japan

🔴 Running Unoptimized Version...
After Start Memory Useage: {'cpu_memory_mb': 630.16, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 1.74, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized
After End Memory Useage: {'cpu_memory_mb': 638.34, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.94, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized

🟢 Running Optimized Version...
After Start Memory Useage: {'cpu_memory_mb': 638.34, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.94, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized
After End Memory Useage: {'cpu_memory_mb': 638.34, 'cpu_usage_percent': 65.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.94, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized

🔎 Results Comparison:
Metric              Unoptimized         Optimized
-------------------------------------

# Optimization 5: Memory and State Management

In [ ]:
memory = ["Hi", "I am roy", "I live in tokyo", "I like lions", "I have five kids"]

def agent_unoptimized(user_input):
    memory.append(user_input)
    prompt = f"{' '.join(memory)}"
    return ollama_chat(model="phi4", prompt=prompt)

def agent_optimized(user_input):
    memory.append(user_input)
    if len(memory) > 5:
        memory.pop(0)  # Keep only last 5 messages
    prompt = f"{' '.join(memory)}"
    return ollama_chat(model="phi4", prompt=prompt)


compare_agents(agent_unoptimized, agent_optimized, "Who am I?", "Optimization 5: Memory and State Management")



===== Optimization 5: Memory and State Management =====
Input: Who am I?

🔴 Running Unoptimized Version...
After Start Memory Useage: {'cpu_memory_mb': 641.64, 'cpu_usage_percent': 19.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized
After End Memory Useage: {'cpu_memory_mb': 641.64, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized

🟢 Running Optimized Version...
After Start Memory Useage: {'cpu_memory_mb': 641.64, 'cpu_usage_percent': 4.8, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized
After End Memory Useage: {'cpu_memory_mb': 641.64, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized

🔎 Results Comparison:
Metric              Unoptimized         Optimized
--------------------------

# Optimization 6: Function Calling Efficiency

In [ ]:
def agent_unoptimized(task):
    prompt = f"What function should I call to {task}?"
    return ollama_chat(model="phi4", prompt=prompt)

def agent_optimized(task):
    prompt = f"""Given the task, choose one function only:
- search()
- get_weather()
- book_flight()

Task: {task}
Return only the function name."""
    return ollama_chat(model="phi4", prompt=prompt)


compare_agents(agent_unoptimized, agent_optimized, "ask my travel agency to book", "Optimization 6: Function Calling Efficiency")



===== Optimization 6: Function Calling Efficiency =====
Input: ask my travel agency to book

🔴 Running Unoptimized Version...
After Start Memory Useage: {'cpu_memory_mb': 641.64, 'cpu_usage_percent': 71.4, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized
After End Memory Useage: {'cpu_memory_mb': 641.64, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized

🟢 Running Optimized Version...
After Start Memory Useage: {'cpu_memory_mb': 641.64, 'cpu_usage_percent': 75.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized
After End Memory Useage: {'cpu_memory_mb': 641.64, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized

🔎 Results Comparison:
Metric              Unoptimized         Optimized
------

# Optimization 7: Fallback Strategies

In [ ]:
def agent_unoptimized(input_text):
    prompt = f"{input_text}"
    return ollama_chat(model="phi4", prompt=prompt)

def agent_optimized(input_text):
    prompt = f"{input_text}"
    result = ollama_chat(model="phi4", prompt=prompt)
    if not result:
        # fallback prompt
        prompt2 = f"Retry: {input_text}. Be concise."
        result = ollama_chat(model="phi4", prompt=prompt2)
    return result

compare_agents(agent_unoptimized, agent_optimized, "I dont have a yellow pen. Now give me my yellow pen.", "Optimization 7: Fallback Strategies")


===== Optimization 7: Fallback Strategies =====
Input: I dont have a yellow pen. Now give me my yellow pen.

🔴 Running Unoptimized Version...
After Start Memory Useage: {'cpu_memory_mb': 641.62, 'cpu_usage_percent': 57.9, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized
After End Memory Useage: {'cpu_memory_mb': 641.62, 'cpu_usage_percent': 5.3, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized

🟢 Running Optimized Version...
After Start Memory Useage: {'cpu_memory_mb': 641.62, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized
After End Memory Useage: {'cpu_memory_mb': 641.62, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized

🔎 Results Comparison:
Metric              Unoptimized         O

# Optimization 8: System Instruction Tuning

In [ ]:
def agent_unoptimized(input_text):
    prompt = f"{input_text}"
    return ollama_chat(model="phi4", prompt=prompt)

def agent_optimized(input_text):
    system = "You are a helpful, concise assistant. Answer precisely."
    prompt = f"{system}\nUser: {input_text}\nAnswer:"
    return ollama_chat(model="phi4", prompt=prompt)

compare_agents(agent_unoptimized, agent_optimized, "I dont have a yellow pen. Now give me my yellow pen.", "Optimization 8: System Instruction Tuning")


===== Optimization 8: System Instruction Tuning =====
Input: I dont have a yellow pen. Now give me my yellow pen.

🔴 Running Unoptimized Version...
After Start Memory Useage: {'cpu_memory_mb': 641.62, 'cpu_usage_percent': 4.8, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized
After End Memory Useage: {'cpu_memory_mb': 641.62, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_unoptimized

🟢 Running Optimized Version...
After Start Memory Useage: {'cpu_memory_mb': 641.62, 'cpu_usage_percent': 14.3, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized
After End Memory Useage: {'cpu_memory_mb': 641.62, 'cpu_usage_percent': 0.0, 'gpu_memory_mb': 0.0, 'gpu_usage_percent': 71.95, 'gpu_total_memory_mb': 15360.0} MB, BY AGENT agent_optimized

🔎 Results Comparison:
Metric              Unoptimized    